## Using a Ridge Regression Machine Learning Model to make Predictions

In [32]:
import pandas as pd
from sklearn.linear_model import Ridge

In [33]:
ml_df = pd.read_csv("CSV/merged_stats_scores.csv")
del ml_df["Unnamed: 0"]

# combined_df of per_game, advanced stats and playoff scores ready for Machine Learning
ml_df

,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,...,TOV%,ORB%,FT/FGA,eFG%_opp,TOV%_opp,DRB%,FT/FGA_opp,Year,Team,Score
0,0.122449,0.676259,0.551724,0.747573,0.312925,0.239474,0.847561,0.787879,0.706044,0.487013,...,0.395062,1.000000,0.387435,0.418440,0.666667,0.438503,0.349462,1996,CHI,10.0
1,0.489796,0.482014,0.297414,0.766990,0.346939,0.318421,0.609756,0.553030,0.458791,0.636364,...,0.777778,0.621053,0.759162,0.397163,0.735632,0.352941,0.510753,1996,OKC,7.0
2,0.428571,0.597122,0.422414,0.786408,0.380952,0.334211,0.695122,0.636364,0.521978,0.623377,...,0.407407,0.584211,0.465969,0.482270,0.425287,0.315508,0.354839,1996,ORL,4.0
3,0.244898,0.532374,0.306034,0.844660,0.176871,0.131579,0.658537,0.795455,0.659341,0.571429,...,0.518519,0.721053,0.696335,0.468085,0.551724,0.470588,0.741935,1996,UTA,4.0
4,0.061224,0.769784,0.737069,0.669903,0.741497,0.710526,0.670732,0.416667,0.329670,0.714286,...,0.506173,0.847368,0.450262,0.631206,0.689655,0.224599,0.322581,1996,ATL,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,0.244898,0.539568,0.737069,0.320388,0.693878,0.776316,0.408537,0.227273,0.260989,0.558442,...,0.382716,0.100000,0.167539,0.773050,0.160920,0.764706,0.209677,2022,ORL,0.0
797,0.122449,0.553957,0.685345,0.398058,0.727891,0.773684,0.500000,0.204545,0.230769,0.597403,...,0.382716,0.263158,0.235602,0.964539,0.275862,0.748663,0.349462,2022,POR,0.0
798,0.306122,0.697842,0.728448,0.572816,0.639456,0.678947,0.487805,0.454545,0.357143,0.707792,...,0.320988,0.178947,0.314136,0.907801,0.183908,0.700535,0.102151,2022,SAC,0.0
799,0.306122,0.892086,0.926724,0.640777,0.632653,0.647368,0.536585,0.674242,0.516484,0.688312,...,0.148148,0.305263,0.120419,0.773050,0.206897,0.652406,0.102151,2022,SAS,0.0


**Indicating which columns the model will be trained on**

In [34]:
not_features = ["Year", "Team", "Score"]

features = ml_df.columns[~ml_df.columns.isin(not_features)]

In [35]:
features

Index(['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'MOV', 'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr',
       '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'eFG%_opp', 'TOV%_opp',
       'DRB%', 'FT/FGA_opp'],
      dtype='object')

In [36]:
features = list(ml_df[features])
features

['MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'MOV',
 'SOS',
 'SRS',
 'ORtg',
 'DRtg',
 'NRtg',
 'Pace',
 'FTr',
 '3PAr',
 'TS%',
 'eFG%',
 'TOV%',
 'ORB%',
 'FT/FGA',
 'eFG%_opp',
 'TOV%_opp',
 'DRB%',
 'FT/FGA_opp']

In [37]:
seasons = list(range(1996, 2023))

## Determine which alpha value to use (to control the regularization component)

#### Some Notes About Cross Validation: <br>

**Cross Validation is a method to assess the performance and generalization ability of a machine learning predictive model. It helps estimate how well a model will perform on unseen data**<br>

**Cross Validation involves dividing the available data into multiple folds or subsets. For 10-fold cross validation, one fold is used as the test set, the other 9 make up the training set. The number of indexes/rows in each fold is simply an equal division of the number of rows in the entire dataset and the number of folds (e.g. 800 rows in total data and 10 folds -> 80 rows per fold). The model is trained on the training fold and used to predict on the training set. For 10-fold cross validation, this process is repeated 9 more times and the results from each iteration are averaged/aggreagate to see the model's overall performance.**<br>

In [38]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# Range of alpha values
# np.logspace(-4, 4, num=9) generates 9 values within the range of 10^(-4) and 10^4
alphas = np.logspace(-4, 4, num=9)

# splits the dataset into 10 folds for performing 10-fold cross-validation. nsplits=10 specifies the number of folds (times the dataset will be divided)
# into training and validation sets

# A fold is simply a subset of the data (number of indexes/rows that are in each fold is the total data divided equally among the number of folds)
# that is used as the cross-validation set exactly once

# Each fold is used once as validation while the 9 other folds form the training set
kf = KFold(n_splits=10)

best_alpha = None

# best_mse is set to infinity to ensure any future MSE value encountered will be lower
best_mse = np.inf

### The following for loop determines the optimal alpha value (to control regularization) for our Ridge Regression Model.

# loops through the values in alphas
for alpha_val in alphas:
    mse_scores = []
    
    # kf.split(ml_df) splits in the indices of the rows in ml_df into train/test folds, returns the indexes of the rows of ml_df divided into
    # two lists
    for train_index, test_index in kf.split(ml_df):
        print(train_index)
        print(test_index)

        # Select the rows denoted by the train_index to act as the training data set
        # .iloc[] selects the rows denoted by the indexes of train_index/test_index
        train = ml_df.iloc[train_index]
        test = ml_df.iloc[test_index]

        # sets the ridge_model with the alpha_val of the current iteration
        ridge_model = Ridge(alpha=alpha_val)

        # fit the model with the training features and the training score
        ridge_model = ridge_model.fit(train[features], train["Score"])

        # use the model to predict the test["Score"] columns given the test features
        predictions = ridge_model.predict(test[features])

        # calculates the mean squared error given the expected and predicted "Score" values
        mse = mean_squared_error(test["Score"], predictions)
        
        # adds the value to mse_scores
        mse_scores.append(mse)
    
    # calculates the mean value of all the mse's in the mse_scores list corresponding to the alpha value of the current iteration
    # mse_scores should have 10 values each one from one of the 10 cross validation folds
    avg_mse = np.mean(mse_scores)
    #print(mse_scores)
    #print(avg_mse)

    # take the alpha value that corresponds with the lowest mse value
    if avg_mse < best_mse:
        best_mse = avg_mse
        best_alpha = alpha_val

best_alpha





[ 81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278
 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296
 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314
 315 316 317 318 319 320 321 322 323 324 325 326 32

10.0

In [39]:
# backtesting function to use the previous years to predict each successive year

def backtest(ml_data, features, score, start=2, step=1):
    total_predictions = []

    for i in range(start, len(seasons), step):
        season = seasons[i]

        # alpha value controls how much regularization the model uses
        ridge_model = Ridge(alpha=10)

        # dividing the ml_data into training and testing data
        train_set = ml_data[ml_data["Year"] < season]

        test_set = ml_data[ml_data["Year"] == season]

        # fits the ridge regression model: adjusts the model paramters to minimize difference between 
        # the actual and predicted output
        ridge_model = ridge_model.fit(train_set[features], train_set[score])

        # Use the model to predict the output given a set of inputs
        predictions = ridge_model.predict(test_set[features])

        # Converting the predictions to a pandas series and joining that with part of the test_set dataframe
        predictions = pd.Series(predictions, index=test_set.index)

        joined_df = pd.concat([test_set[["Team", "Score", "Year"]], predictions], axis=1)

        # Renaming columns of the new dataframe
        joined_df.columns = ["Team", "Actual_Score", "Year", "Predicted_Score"]

        total_predictions.append(joined_df)

    # this joins all the DataFrames in total_predictions along axis=0 (vertical stacking)
    return pd.concat(total_predictions)


In [40]:
results = backtest(ml_df, features, "Score")

In [41]:
# Sorting data from oldest to latest season and then sorting based on who was predicted the 
# highest playoff score

results = results.sort_values(["Year", "Predicted_Score"], ascending=[True, False])

In [42]:
results.to_csv("PREDICTIONS/results_ridge.csv")

## Seeing how well the model did

**Calculating Mean Squared Error**

In [43]:
from sklearn.metrics import mean_squared_error

In [44]:
ridge_df = pd.read_csv("PREDICTIONS/results_ridge.csv")
del ridge_df["Unnamed: 0"]

In [45]:
ridge_df

,Team,Actual_Score,Year,Predicted_Score
0,LAL,4.0,1998,2.986818
1,OKC,2.0,1998,2.898816
2,UTA,7.0,1998,2.730749
3,PHO,1.0,1998,2.581946
4,CHI,10.0,1998,2.549527
...,...,...,...,...
738,HOU,0.0,2022,-0.462144
739,OKC,0.0,2022,-0.655847
740,ORL,0.0,2022,-0.757063
741,DET,0.0,2022,-0.951902


In [46]:
mean_squared_error(ridge_df["Actual_Score"], ridge_df["Predicted_Score"])

3.024286621590587

**Some further analysis**

In [47]:
ridge_df
#del ridge_df["Actual_Rank"]
#del ridge_df["Predicted_Rank"]

,Team,Actual_Score,Year,Predicted_Score
0,LAL,4.0,1998,2.986818
1,OKC,2.0,1998,2.898816
2,UTA,7.0,1998,2.730749
3,PHO,1.0,1998,2.581946
4,CHI,10.0,1998,2.549527
...,...,...,...,...
738,HOU,0.0,2022,-0.462144
739,OKC,0.0,2022,-0.655847
740,ORL,0.0,2022,-0.757063
741,DET,0.0,2022,-0.951902


In [48]:
ridge_df

,Team,Actual_Score,Year,Predicted_Score
0,LAL,4.0,1998,2.986818
1,OKC,2.0,1998,2.898816
2,UTA,7.0,1998,2.730749
3,PHO,1.0,1998,2.581946
4,CHI,10.0,1998,2.549527
...,...,...,...,...
738,HOU,0.0,2022,-0.462144
739,OKC,0.0,2022,-0.655847
740,ORL,0.0,2022,-0.757063
741,DET,0.0,2022,-0.951902


**Indicating the position/rank each team actually ended up in based on their actual playoff score**<br>
**NOTE: This ranking isn't enirely accurate as the teams that missed the playoffs entirely has a score of 0 so technically they're equal**<br>
**which doesn't reflect the ranking**

In [49]:
# add_pos_rank(df, type_rk) adds either a Actual or Predicted "Rank" column that reflects the Team's likelihood to win the NBA Championship
# Actual Rank: based on the Team's "Actual Playoff Score" and shows the team that won it all, got eliminated in the finals, conference finals, etc.
# Note: Actual Rank enirely accurate as the teams that missed the playoffs entirely has a score of 0 so technically they're equal w
# which doesn't reflect the ranking

# Predicted Rank: based on the "Team's Predicted Score" and reflects the team's likelihood to win the NBA Championship
def add_pos_rank(df, type_rk):
    if type_rk == "Actual_Rank":
        df = df.sort_values(["Actual_Score"], ascending=False)

        # Adds an "Actual Rank" which goes from 1 to 30
        actual_rank = list(range(1, df.shape[0] + 1))
        df.insert(2, "Actual_Rank", actual_rank)
    elif type_rk == "Predicted_Rank":
        df = df.sort_values(["Predicted_Score"], ascending=False)

        # Adds a "Predicted Rank" which goes from 1 to 30
        predicted_rank = list(range(1, df.shape[0] + 1))
        df.insert(5, "Predicted_Rank", predicted_rank)
    return df

In [50]:
# Groups the data by year and applies add_pos_rank with the Actual Rank argument to each group (i.e. season)
# So the ranks (1-30) are applied to every single season

ridge_df = ridge_df.groupby("Year").apply(add_pos_rank, type_rk = "Actual_Rank")

In [51]:
# Drops an extra index level caused by the groupby method

ridge_df.index = ridge_df.index.droplevel()
ridge_df

,Team,Actual_Score,Actual_Rank,Year,Predicted_Score
4,CHI,10.0,1,1998,2.549527
2,UTA,7.0,2,1998,2.730749
0,LAL,4.0,3,1998,2.986818
6,IND,4.0,4,1998,2.182407
1,OKC,2.0,5,1998,2.898816
...,...,...,...,...,...
732,NYK,0.0,26,2022,1.172252
729,CHO,0.0,27,2022,1.459328
727,LAC,0.0,28,2022,1.502158
719,CLE,0.0,29,2022,2.350557


In [52]:
# Renaming a column in a pandas dataframe
# ridge_df = ridge_df.rename(columns={"Actual Rank": "Actual_Rank"})

In [53]:
# Reordering columns in a pandas dataframe
# ridge_df = ridge_df[["Year", "Team", "Actual_Score", "Predicted_Score", "Actual_Rank"]]

**Indicating the position/rank each team was predicted to end up in based on their predicted playoff score**

In [54]:
# Groups the data by year and applies add_pos_rank with the Predicted Rank argument to each group (i.e. season)
# So the ranks (1-30) are applied to every single season
 
ridge_df = ridge_df.groupby("Year").apply(add_pos_rank, type_rk = "Predicted_Rank")

# Drops an extra index caused by the groupby method
ridge_df.index = ridge_df.index.droplevel()
ridge_df

,Team,Actual_Score,Actual_Rank,Year,Predicted_Score,Predicted_Rank
0,LAL,4.0,3,1998,2.986818,1
1,OKC,2.0,5,1998,2.898816,2
2,UTA,7.0,2,1998,2.730749,3
3,PHO,1.0,15,1998,2.581946,4
4,CHI,10.0,1,1998,2.549527,5
...,...,...,...,...,...,...
738,HOU,0.0,21,2022,-0.462144,26
739,OKC,0.0,19,2022,-0.655847,27
740,ORL,0.0,17,2022,-0.757063,28
741,DET,0.0,20,2022,-0.951902,29


In [55]:
# Re-ordering the columns

ridge_df = ridge_df[["Year", "Team", "Actual_Score", "Predicted_Score", "Actual_Rank", "Predicted_Rank"]]
ridge_df

,Year,Team,Actual_Score,Predicted_Score,Actual_Rank,Predicted_Rank
0,1998,LAL,4.0,2.986818,3,1
1,1998,OKC,2.0,2.898816,5,2
2,1998,UTA,7.0,2.730749,2,3
3,1998,PHO,1.0,2.581946,15,4
4,1998,CHI,10.0,2.549527,1,5
...,...,...,...,...,...,...
738,2022,HOU,0.0,-0.462144,21,26
739,2022,OKC,0.0,-0.655847,19,27
740,2022,ORL,0.0,-0.757063,17,28
741,2022,DET,0.0,-0.951902,20,29


In [56]:
ridge_df.to_csv("PREDICTIONS/results_ridge.csv")

### Predicting the Champion for the 2023 NBA Season

In [57]:
season = 2023

In [58]:
# Reading in the 2023 Team Per Game and Advanced Stats data
data_2023 = pd.read_csv("2023_Predictions/2023_team_stats.csv")
del data_2023["Unnamed: 0"]
data_2023

,Team,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,TS%,eFG%,TOV%,ORB%,FT/FGA,eFG%_opp,TOV%_opp,DRB%,FT/FGA_opp,Year
0,SAC,0.387097,0.814815,0.500000,0.80,0.548387,0.587413,0.700000,0.549451,0.410811,...,1.000000,0.982456,0.447368,0.385246,0.6875,0.767857,0.448980,0.793651,0.507937,2023
1,GSW,0.387097,0.722222,0.727273,0.50,1.000000,1.000000,0.966667,0.186813,0.200000,...,0.862069,0.964912,1.000000,0.524590,0.1000,0.357143,0.387755,0.603175,0.682540,2023
2,ATL,0.483871,1.000000,0.977273,0.58,0.064516,0.111888,0.416667,1.000000,1.000000,...,0.500000,0.438596,0.236842,0.581967,0.3750,0.571429,0.408163,0.571429,0.555556,2023
3,BOS,1.000000,0.555556,0.568182,0.42,0.903226,0.958042,0.833333,0.153846,0.156757,...,0.862069,0.877193,0.447368,0.336066,0.3375,0.142857,0.183673,1.000000,0.142857,2023
4,OKC,0.483871,0.722222,1.000000,0.22,0.274194,0.363636,0.483333,0.681319,0.821622,...,0.344828,0.263158,0.236842,0.549180,0.4625,0.482143,0.816327,0.111111,0.809524,2023
5,LAL,0.580645,0.685185,0.590909,0.56,0.064516,0.160839,0.316667,0.802198,0.783784,...,0.551724,0.456140,0.526316,0.393443,0.7750,0.267857,0.102041,0.650794,0.000000,2023
6,UTA,0.290323,0.611111,0.681818,0.38,0.467742,0.622378,0.433333,0.483516,0.470270,...,0.586207,0.543860,0.789474,0.721311,0.4875,0.375000,0.102041,0.476190,0.539683,2023
7,MEM,0.193548,0.833333,0.943182,0.42,0.258065,0.370629,0.400000,0.758242,0.789189,...,0.344828,0.421053,0.368421,0.696721,0.2500,0.107143,0.551020,0.587302,0.555556,2023
8,MIL,0.387097,0.648148,0.750000,0.38,0.709677,0.797203,0.683333,0.340659,0.367568,...,0.568966,0.684211,0.631579,0.573770,0.1750,0.000000,0.000000,0.888889,0.063492,2023
9,IND,0.096774,0.518519,0.659091,0.30,0.516129,0.566434,0.666667,0.395604,0.502703,...,0.534483,0.508772,0.710526,0.442623,0.4875,0.607143,0.530612,0.000000,0.920635,2023


In [59]:
# Combining the ml_df (which contains each Team's Per Game and Advanced Stats from 1996-2022) with the 2023 Data
pred_2023 = pd.concat([ml_df, data_2023], axis=0)
pred_2023

,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,...,TOV%,ORB%,FT/FGA,eFG%_opp,TOV%_opp,DRB%,FT/FGA_opp,Year,Team,Score
0,0.122449,0.676259,0.551724,0.747573,0.312925,0.239474,0.847561,0.787879,0.706044,0.487013,...,0.395062,1.000000,0.387435,0.418440,0.666667,0.438503,0.349462,1996,CHI,10.0
1,0.489796,0.482014,0.297414,0.766990,0.346939,0.318421,0.609756,0.553030,0.458791,0.636364,...,0.777778,0.621053,0.759162,0.397163,0.735632,0.352941,0.510753,1996,OKC,7.0
2,0.428571,0.597122,0.422414,0.786408,0.380952,0.334211,0.695122,0.636364,0.521978,0.623377,...,0.407407,0.584211,0.465969,0.482270,0.425287,0.315508,0.354839,1996,ORL,4.0
3,0.244898,0.532374,0.306034,0.844660,0.176871,0.131579,0.658537,0.795455,0.659341,0.571429,...,0.518519,0.721053,0.696335,0.468085,0.551724,0.470588,0.741935,1996,UTA,4.0
4,0.061224,0.769784,0.737069,0.669903,0.741497,0.710526,0.670732,0.416667,0.329670,0.714286,...,0.506173,0.847368,0.450262,0.631206,0.689655,0.224599,0.322581,1996,ATL,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,0.193548,0.240741,0.284091,0.320000,0.064516,0.153846,0.316667,0.549451,0.643243,0.328571,...,0.815789,0.475410,0.712500,0.535714,0.551020,0.873016,0.634921,2023,ORL,NaN
26,0.387097,0.388889,0.750000,0.060000,0.048387,0.251748,0.050000,0.626374,0.789189,0.171429,...,0.526316,0.475410,0.312500,0.428571,0.428571,0.523810,0.634921,2023,CHO,NaN
27,0.096774,0.259259,0.579545,0.060000,0.000000,0.209790,0.000000,0.593407,0.735135,0.200000,...,0.973684,1.000000,0.562500,0.785714,0.285714,0.571429,0.746032,2023,HOU,NaN
28,0.290323,0.074074,0.375000,0.000000,0.161290,0.244755,0.400000,0.373626,0.610811,0.000000,...,0.789474,0.565574,0.712500,0.660714,0.306122,0.285714,0.952381,2023,DET,NaN


In [60]:
# alpha value controls how much regularization the model uses
ridge_model = Ridge(alpha=10)

# dividing the pred_2023 data into training and testing data
train_set = pred_2023[pred_2023["Year"] < season]
test_set = pred_2023[pred_2023["Year"] == season]

# fits the ridge regression model: adjusts the model paramters to minimize difference between 
# the actual and predicted output
ridge_model = ridge_model.fit(train_set[features], train_set["Score"])

# Use the model to predict the output given a set of inputs
predictions = ridge_model.predict(test_set[features])

# Converting the predictions to a Pandas series
predictions = pd.Series(predictions, index=test_set.index)

# joining the predictions Pandas series with the Team and Year columns of the test_set dataframe
joined_df_2023 = pd.concat([test_set[["Team", "Year"]], predictions], axis=1)

# Renaming columns of the new dataframe
joined_df_2023.columns = ["Team", "Year", "Predicted_Score"]

**Indicating the position/rank each team was predicted to end up in based on their predicted playoff score**

In [61]:
# Sorting the joined_df_2023 DataFrame by the Predicted Playoff Score in descending order
joined_df_2023 = joined_df_2023.sort_values(["Predicted_Score"], ascending=False)

# Adding a Predicted Rank which is the likelihood the team won the NBA Championship based on the Predicted Playoff Score
predicted_rank = list(range(1, joined_df_2023.shape[0] + 1))

# Gets the number of columns of the DataFrame
new_col = joined_df_2023.shape[1] 

# Inserting the Predicted Rank column as the last column in the DataFrame
joined_df_2023.insert(new_col, "Predicted_Rank", predicted_rank)

In [62]:
# turning the DataFrame into a csv file

joined_df_2023.to_csv("PREDICTIONS/predictions_2023_NBA_Champion.csv")

### Some Machine Learning Terms<br>

**Sum of Residual Squares: sum of the difference between predicted value and actual value squared for each data point.**<br><br>
**Bias: inability for a machine learning method (like linear regression) to capture the true relationship of a dataset.**<br>
**Larger the bias, the less accurate the model can capture the true relationship.**<br>
**Smaller the bias, the more accurate the mode can capture the true relationship.**<br><br>
**Variance: how well the model (predicted values) fits the expected data values which is quantified by the Sum of Residual Squares. GENERALLY, the lower the Sum of Residual Squares, the lower the variance indicating a better-fitting model. The higher the Sum of Residual Squares, the higher the variance and the worse the model fits the data.**

**Ridge Regression works by introducing some bias to get better variance and prevent overfitting.**<br>

**Overfitting: when the model works really well on the training data, but is very inaccurate on the testing data.**<br>

### Some Ridge Regression Notes<br>

**Alpha value represents the amount of "shrinkage" that will be used in the Ridge Regression equation to control the regularization of the model**<br>
**Ridge Regression tries to minimize the RSS (Residual Sum of Squares) + alpha * (slope)^2**